In [ ]:
# !pip install selenium

In [43]:
import numpy as np
import pandas as pd
import time 
import datetime
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [44]:
# driver.get('https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/2019-6-24?cm_ven=localwx_history')

In [ ]:
# weatherdata = driver.find_elements_by_id('history-observation-table')[0].text

In [ ]:
# weatherdata

In [ ]:
# var = 10
# month_element = driver.find_element_by_xpath('//*[@id="inner-content"]/div[2]/div[1]/div/div[1]/div/div/date-selector/div/select[1]/option[%s]'% var)
# month_element.click()

In [61]:
# def weather_table(month,day, selected_year, current_year): #month: alphabetic, rest are numeric
 
#     month_dict = {'January':1,
#              'February':2,
#              'March':3,
#              'April':4,
#              'May':5,
#              'June':6,
#              'July':7,
#              'August':8,
#              'September':9,
#              'October':10,
#              'November':11,
#              'December':12}

#     m = month_dict.get(month)
#     d = int(day)
#     y = int(current_year) - int(selected_year) + 1

#     month_element = driver.find_element_by_xpath('//*[@id="inner-content"]/div[2]/div[1]/div/div[1]/div/div/date-selector/div/select[1]/option[%s]' % m).click()
#     day_element = driver.find_element_by_xpath('//*[@id="inner-content"]/div[2]/div[1]/div/div[1]/div/div/date-selector/div/select[2]/option[%s]' % d).click()
#     year_element = driver.find_element_by_xpath('//*[@id="inner-content"]/div[2]/div[1]/div/div[1]/div/div/date-selector/div/select[3]/option[%s]' % y).click()
#     view = driver.find_element_by_xpath('//*[@id="inner-content"]/div[2]/div[1]/div/div[1]/div/div/date-selector/div/input')
#     view.click()
    
# #     print(len(driver.find_elements_by_id('history-observation-table')))
#     weatherdata = driver.find_elements_by_id('history-observation-table')[0].text
#     header,info = weatherdata.split('\n',1)
#     info_lst = info.split(' in')
    
#     header = header.split(' ')[:2]
#     filtered = [x for x in info_lst if len(x) > 4]

#     time_lst = []
#     temp_lst = []

#     for i in range(len(filtered)):
#         if i == 0:
#             time = filtered[i].split('\n')[0]
#             temp = int(filtered[i].split('\n')[1].split(' ')[0])
#         else:
#             time = filtered[i].split('\n')[1]
#             temp = int(filtered[i].split('\n')[2].split(' ',1)[0])

#         time_lst.append(time)
#         temp_lst.append(temp)

#     df = pd.DataFrame(list(zip(time_lst,temp_lst)),columns = header) 
#     df['Date'] = str(month) + '-'+ str(day)+'-' + str(selected_year)
    
#     weatherdata = ''
    
#     return df
    
    
    

In [47]:
def weather_table(date): #year-month-date
    driver = webdriver.Chrome('/Users/duytienpham/eclipse-workspace/ChromeDriver/chromedriver')
    driver.get('https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/{}'.format(date))
    driver.implicitly_wait(15)

    time_lst = []
    temp_lst = []

    rows = driver.find_element_by_xpath('//*[@id="history-observation-table"]/tbody').find_elements_by_tag_name('tr')
    for row in rows:
        time_lst.append(row.find_elements_by_tag_name('td')[0].text)
        temp_lst.append(row.find_elements_by_tag_name('td')[1].text)
        
    df = pd.DataFrame(list(zip(time_lst,temp_lst)), columns = ['Time','Temperature']) 
    df['Date'] = date
    driver.quit() #important - must always quit browser in order to avoid parallel usage selenium
   
    return df

In [49]:
def wunderground_scraping(start,end):
    ''' Enter a string of date which format is year-month-date (numeric)
    ex: '2017-11-27'
    '''
    start_year = int(start.split('-')[0])
    start_month = int(start.split('-')[1])
    start_date = int(start.split('-')[-1])
#     start = '-'.join([str(start_year),str(start_month),str(start_date)])
    
    end_year = int(end.split('-')[0])
    end_month = int(end.split('-')[1])
    end_date = int(end.split('-')[-1])
#     end = '-'.join([str(end_year),str(end_month),str(end_date)])
    
    day_range = list(range(32))
#     month_range = ['January','February','March','April','May','June','July','August','September','October','November','December']
    month_range = list(range(1,13))
    month_length = {1:31,
             2:28,
             3:31,
             4:30,
             5:31,
             6:30,
             7:31,
             8:31,
             9:30,
             10:31,
             11:30,
             12:31}
    
    
    master_df = pd.DataFrame()
#     start_month_idx = month_range.index(start_month)
#     end_month_idx = month_range.index(end_month)
    
    if start_month == end_month:
        for i in range(start_date, end_date+1):
            dat = '-'.join([str(start_year),str(start_month), str(i)])
            df = weather_table(dat)
            master_df = pd.concat([master_df,df])
    else:
        for month in range(start_month, end_month+1):
            if month == start_month:
                for i in range(start_date, month_length.get(month) + 1):
                    dat = '-'.join([str(start_year),str(start_month), str(i)])
                    df = weather_table(dat)
                    master_df = pd.concat([master_df,df])
                    
            elif month == end_month:
                for i in range(1, end_date+1):
                    dat = '-'.join([str(end_year),str(end_month), str(i)])
                    df = weather_table(dat)
                    master_df = pd.concat([master_df,df]) 
                    
            else:
#                 (month != start_month) and (month != end_month):
                for i in range(1, month_length.get(month) + 1):
                    dat = '-'.join([str(start_year),str(month), str(i)])
                    df = weather_table(dat)
                    master_df = pd.concat([master_df,df]) 
                
    return master_df     

In [32]:
data47_18 = wunderground_scraping('2018-4-1','2018-7-31')

In [33]:
data810_18 = wunderground_scraping('2018-8-1','2018-10-31')

In [34]:
data2018 = pd.concat([data47_18,data810_18])

In [37]:
data2018 = data2018.reset_index()
data2018 = data2018.drop(columns = ['index'])
# data2018.to_csv('temp2018.cvs')

In [62]:
data2018.to_csv('temp2018.csv')

In [54]:
data47_17 = wunderground_scraping('2017-4-1','2017-10-31')

In [58]:
data47_17 = data47_17.reset_index()
data47_17.columns

Index(['level_0', 'index', 'Time', 'Temperature', 'Date'], dtype='object')

In [59]:
data47_17 = data47_17.drop(columns = ['level_0','index'])

In [63]:
data46_18 = wunderground_scraping('2019-4-1','2019-6-25')

In [64]:
data46_18.to_csv('temp_2019.csv')